In [4]:
import requests
from bs4 import BeautifulSoup

In [91]:
class UrlGetter:
    def __init__(self):
        pass
    
    def get_afp_url(self, page):
        return "http://www.afpbb.com/category/news?page={}".format(page)
    
    def get_reuters_url(self, page):
        return "https://jp.reuters.com/news/archive/topNews?view=page&page={}&pageSize=10".format(page)
    
    def get_cnn_url(self, category, page):
        return "https://www.cnn.co.jp/{}/{}/".format(category, page)
        
    def get_url(self, site_type, *params):
        return getattr(self, "get_{}_url".format(site_type))(*params)

In [92]:
class ArticleGetter:
    def __init__(self, base_url):
        self.base_url = base_url
        r = requests.get(base_url)
        self.soup = BeautifulSoup(r.content, "html.parser")
    
    def get_afp_articles(self):
        ul = self.soup.find("ul", {"id": "tab-latest"})
        return list(set([result['href'] for result in ul.find_all("a")]))
    
    def get_cnn_articles(self):
        results = []
        url_list = self.soup.find("ul", attrs={"class":"list-news-line"})
        return ["https://www.cnn.co.jp"+result['href'] for result in set(url_list.find_all("a"))]
    
    def get_reuters_articles(self):
        results = []
        for tmp_result in self.soup.find_all("div", attrs={"class":"story-content"}):
            results += tmp_result.find_all("a")
        return ["https://jp.reuters.com"+result['href'] for result in set(results)]
    
    def get_articles(self, site_type):
        return getattr(self, "get_{}_articles".format(site_type))()

In [110]:
import re
class ArticleExtractor:
    def __init__(self, target_url):
        self.target_url = target_url
        r = requests.get(target_url)
        self.soup = BeautifulSoup(r.content, "html.parser")
        [s.extract() for s in self.soup('script')]
        [s.extract() for s in self.soup('link')]
        
    def extract_afp_article(self):
        title = self.soup.find("h1", {"class": "title"}).get_text()
        body = self.soup.find("div", {"class": "article-body"}).get_text()
        body = re.sub(r"【.+?】", "", body)
        body = re.sub(r"\(c\)AFP.*", "", body)
        body = body.replace("AFP","")
        day  = self.soup.find("span", {"class": "day"})
        day.find("span").extract()
        return title, body, day.get_text()
    
    def extract_reuters_article(self):
        article = self.soup.find("div", attrs={"id": self.target_url.split("-id")[1]})
        title = article.find("h1").get_text()
        tmp = article.find("h1")
        tmp.extract()
        tmp = article.find_all(re.compile(r"(div|p|span)"), {"class": re.compile(r"(channel_*|reading-time_*|attribution_*|trustBadge*)")})
        [t.extract() for t in tmp]
        tmp = article.find("div", {"class": re.compile(r"date_*")})
        day = tmp.get_text()
        tmp.extract()
        body = article.get_text()
        body = re.sub(r"［.+?ロイター］","", body)[2:]
        return title, body, day
    
    def extract_cnn_article(self):
        article = self.soup.find("article")
        title = article.find("h1").get_text()
        body = article.find("div", attrs={"id": "leaf-body"}).get_text().replace("（ＣＮＮ）","")
        day = article.find("div", {"class": "metadata-updatetime"}).get_text()
        return title, body, day
    
    def extract_article(self, site_type):
        title, body, day = getattr(self, "extract_{}_article".format(site_type))()
        title = title.replace("\n", " ").replace("\u3000", " ").replace("\r","").strip()
        body = body.replace("\n", " ").replace("\u3000", " ").replace("\r", "").strip()
        day = day.replace("\n", " ").replace("\u3000", " ").replace("\r", "").strip()
        return title, body, day

In [111]:
site_type = "reuters"
urlgetter = UrlGetter()
base_url = urlgetter.get_url(site_type, 1)
target_url = ArticleGetter(base_url).get_articles(site_type)[1]
ArticleExtractor(target_url).extract_article(site_type)

('中国、2015年のような為替介入の意図はない＝関係者',
 '内部関係者によると、中国政府は人民元の下落を心地良くとらえており、市場介入の実施は急激で不安定な下落の阻止や市場の信頼回復を目的とする時のみになるとみている。  ７月４日、内部関係者によると、中国政府は人民元の下落を心地良くとらえており、市場介入の実施は急激で不安定な下落の阻止や市場の信頼回復を目的とする時のみになるとみている。写真は人民元紙幣。昨年５月撮影（２０１８年 ロイター/Thomas White/Illustration）３日の中国市場では、株は下落し人民元は主要な心理的節目の１ドル＝６．７元を突破しだ。市場筋によると、時に人民銀行の代理で行動する国有銀行は人民元押し上げに懸命だった。 それでも当局は、株と為替が急落した２０１５年のように人民元防衛に巨額の外貨準備を活用する必要はないと自信を抱いている。政策決定者の方針に詳しい３人の関係者がロイターに語った。 ２０１５年を教訓として、中国政府は不安定化を招くような人民元の大量売りを阻止する政策はすでに導入済みとみている。 関係者によると、介入は貿易戦争への懸念で揺れる市場に信頼感を与える意図を明確にしたが、景気減速の影響を緩和するための人民元安は容認される見通し。米国の対中関税発動方針の影響を和らげる目的もあるという。 ある内部関係者は「政策決定者はある程度の元安は問題ないと考えている。ただ、１ドル＝６．９元超の元安は望んでいない。経済が下向きの圧力を受けているため、適度な元安は必要だ」と述べた。 別の関係者も同意見で、「元安に大きな問題はない。景気が減速しているため恩恵がある。資本流出もコントロール可能で、積極的な介入の必要はない」と述べた。 中国人民銀行はコメント要請に応じていない。 また、政府が昨年５月に人民元の対ドル基準値算出で導入した「カウンターシクリカル（反景気循環）な調整要素」のような別のツールを採用することも予想されている。 政府に助言を与えている別の関係者は「当局は為替の管理で経験を深めている」と指摘し、６．７─６．８元のエリアでは何らかの介入があるとの見方を示した。',
 'July 5, 2018 /  12:55 AM / an hour ago更新')

In [ ]:
from tqdm import tqdm_notebook
site_type = "afp"
urlgetter = UrlGetter()
results = []
for i in tqdm_notebook(range(3850)):
    base_url = urlgetter.get_url(site_type, i+1)
    for target_url in ArticleGetter(base_url).get_articles(site_type):
        title, body = ArticleExtractor(target_url).extract_article(site_type)
        results.append({
            "title": title,
            "body": body,
            "url": target_url            
        })

In [27]:
import pandas as pd
df = pd.read_csv("afp.csv")

In [37]:
df.iloc[3]['body']

'ベルギーとフランス、ドイツ3か国の当局は、フランスの首都パリ郊外で先月30日に開かれたイラン反体制派組織の集会に対する爆弾攻撃を計画していた疑いで、イランの外交官1人を含む6人の身柄を拘束した。ベルギー当局や複数の情報筋が2日、明らかにした。  ベルギー連邦検察当局の発表によると、攻撃計画があったとされるのはイラン反体制派組織ムジャヒディン・ハルク（イスラム人民戦士機構、MKO）がパリ北郊ビルパント（Villepinte）で開催した集会。約2万5000人が参加した。  ドナルド・トランプ（Donald Trump）米大統領の顧問弁護士を務めるルドルフ・ジュリアーニ（Rudy Giuliani）元ニューヨーク市長を含む、米国の有力者らも出席したという。  容疑者の拘束についてはベルギー連邦当局が最初に発表。検察が「イラン系」のベルギー人とする夫婦に対し、同集会で「爆弾攻撃の実行を企図した疑いが持たれている」と明らかにした。  警察の精鋭部隊がブリュッセルの住宅地で夫婦を呼び止めた際、2人は高性能爆薬「TATP（過酸化アセトン）」500グラムと起爆装置を所持していた。夫婦と接触があったオーストリア・ウィーン駐在のイラン人外交官もドイツで拘束されたという。  治安筋によると、フランスでも先月30日に3人が拘束されたが、うち2人は釈放された。  2日には、イランのハッサン・ロウハニ（Hassan Rouhani）大統領が訪欧。イラン政府は今回の訪問を、米国がイランとの核合意から離脱したことを受けて、イランと欧州との協力のための「決定的重要性」を持つ訪問と位置づけている。'